In [2]:
!pip install tensorflow numpy


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import urllib.request, numpy as np, tensorflow as tf
from tensorflow.keras import layers, callbacks

In [4]:
url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
urllib.request.urlretrieve(url, 'shakespeare.txt')
with open('shakespeare.txt', 'r', encoding='utf-8') as f :
    text = f.read().lower()

chars = sorted(list(set(text)))
print("Chars : \n", chars)
char_to_idx = {c:i for i,c in enumerate(chars)}
idx_to_char = {i:c for i,c in enumerate(chars)}
vocab_size = len(chars)

Chars : 
 ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [5]:
seq_len = 50
step = 3
X, y = [], []
for i in range(0, len(text)-seq_len, step):
    X.append([char_to_idx[c] for c in text[i:i+seq_len]])
    y.append(char_to_idx[text[i+seq_len]])
print("X: \n", X)
print("Y: \n", y)
X = np.array(X)
y = tf.keras.utils.to_categorical(y, vocab_size)
print("X(Transformed): \n", X)
print("Y(Transformed): \n", y)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



X(Transformed): 
 [[18 21 30 ... 17 13 30]
 [31 32  1 ...  1 25 17]
 [15 21 32 ...  1 31 28]
 ...
 [32 33 26 ... 32  1 35]
 [17  1 31 ... 13 23 21]
 [24 17 17 ... 26 19  8]]
Y(Transformed): 
 [[0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [10]:
model = tf.keras.Sequential([
    layers.Embedding(vocab_size, 128, input_length=seq_len),
    layers.LSTM(128, return_sequences=True),
    layers.Dropout(0.2),
    layers.LSTM(128),
    layers.Dense(vocab_size, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

early = callbacks.EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
model.fit(X, y, batch_size=128, epochs=3, callbacks=[early], verbose=1)

loss, acc = model.evaluate(X, y, verbose=0)
print(f"Final Model Accuracy: {acc:.4f}")
print(f"Final Model Loss: {loss:.4f}")

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/3
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 443s 152ms/step - accuracy: 0.2966 - loss: 2.4650
Epoch 2/3
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 592s 204ms/step - accuracy: 0.4435 - loss: 1.8656
Epoch 3/3
2905/2905 ━━━━━━━━━━━━━━━━━━━━ 362s 124ms/step - accuracy: 0.4844 - loss: 1.7135
Final Model Accuracy: 0.5168
Final Model Loss: 1.5929


In [11]:
def generate(seed: str, length: int=200, temperature = 1.0):
    seed_seq = seed.lower()[-seq_len:].ljust(seq_len, '\x00')
    x = np.array([char_to_idx.get(c, 0) for c in seed_seq]).reshape(1,seq_len)
    result = seed

    for _ in range(length):
        probs = model.predict(x, verbose=0)[0]
        probs = np.log(probs + 1e-12) / temperature
        probs = np.exp(probs) / np.sum(np.exp(probs))
        idx = np.random.choice(vocab_size, p=probs)
        result += idx_to_char[idx]
        x = np.roll(x, -1)
        x[0, -1] = idx
    return result
print("\n=== Sample generations ===")
for seed in ["to be or not to be", "the king ", "romeo, romeo, "]:
    print(f"\nSeed : {seed}")
    print("Gen  :", generate(seed, length=120, temperature=0.9))


=== Sample generations ===

Seed : to be or not to be
Gen  : to be or not to beinknio:
that the tounh that with, prosperour's lord,
and sen: jy hast off you; therein with to the alls
the mine,
proces

Seed : the king 
Gen  : the king ism:

grupom:
enecay: leived not o'd-her, have hes this for,
but nid of before, the say the hald honous will comless?

t

Seed : romeo, romeo, 
Gen  : romeo, romeo, itini:
my, here provines--
dugh the be pasch'd fave all poorster, in clatess. and,
how the shall fair not; my foirst in 


In [14]:
 model.build(input_shape=(None, seq_len))
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 50, 128)             │           4,992 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 50, 128)             │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 50, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 39)                  │           5,031 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 819,575 (3.13 MB)

 Trainable params: 273,191 (1.04 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 546,384 (2.08 MB)